In [1]:
# ruff: noqa: F401

In [2]:
%load_ext autoreload
%load_ext pyinstrument

%autoreload 2

In [3]:
import sys

from dotenv import load_dotenv

sys.path.insert(0, "..")

load_dotenv()

True

In [4]:
from datetime import datetime, timedelta
from pathlib import Path

import hvplot.polars
import numpy as np
import polars as pl
from sqlalchemy import text

In [5]:
from tsdb_benchmarks.dbs.questdb import QuestDB

In [6]:
q = QuestDB()

q.fetch("select version()").to_dicts()

[{'version()': 'PostgreSQL 12.3, compiled by Visual C++ build 1914, 64-bit, QuestDB'}]

In [7]:
df = pl.read_parquet("../data/input/time_series/data_wide_0.5M_0.1k.parquet")

df

time,ratio_5,process_17,deviation_2,process_61,process_48,process_64,process_83,process_41,process_50,process_63,binary_1,process_27,process_49,process_22,process_82,ratio_3,process_47,process_33,binary_2,process_29,process_67,process_15,process_76,process_11,process_18,process_37,binary_5,process_23,process_71,process_56,process_36,process_32,ratio_2,process_40,process_46,process_16,…,process_70,process_75,deviation_4,binary_4,process_9,process_21,process_4,process_34,process_62,process_42,process_68,process_14,binary_3,process_28,process_51,process_13,process_78,deviation_5,process_12,process_65,process_39,process_43,process_74,process_35,process_85,process_84,process_55,process_38,deviation_1,process_44,process_30,process_26,process_8,ratio_4,process_77,process_6,process_66
datetime[ms],f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,bool,f32,f32,f32,f32,f32,f32,f32,bool,f32,f32,f32,f32,f32,f32,f32,bool,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,bool,f32,f32,f32,f32,f32,f32,f32,f32,bool,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
2024-01-19 18:41:00,0.80593,300.455414,-7.731582,740.586548,1196.729736,409.302277,245.100708,46.619602,298.892365,230.019775,false,162.484207,210.038422,495.126068,210.16008,0.682768,199.832901,811.640747,true,54.126534,765.163574,162.938828,847.684875,196.484619,720.473694,225.642929,false,619.372437,271.541534,582.658813,576.870789,1104.865356,0.06206,590.350159,331.366028,82.230919,…,145.256012,254.71315,15.457435,false,265.88736,223.479507,662.40979,97.776443,166.551407,266.35614,353.419434,180.686722,true,449.556763,742.759705,124.45755,231.951065,-34.818947,405.773346,116.588463,401.022186,402.66922,414.300262,592.503967,67.806801,774.826721,270.66452,152.048203,18.754868,78.914421,60.698078,323.788177,212.264633,0.127276,293.387909,112.893295,158.082489
2024-01-19 18:42:00,0.452803,300.406464,-7.622905,740.469604,1196.512695,409.482239,245.651321,46.738731,299.173065,230.37178,false,162.648407,210.008469,495.072479,209.9673,0.218096,199.532593,811.500244,true,54.148956,765.016418,162.773697,847.629822,196.699417,720.023438,225.656479,false,620.314941,271.786346,582.508911,576.951965,1104.799438,0.237426,590.403198,331.131775,81.91906,…,145.129303,254.586044,11.870923,false,266.061432,223.640991,662.453491,97.772408,166.528839,265.902649,353.523163,180.776428,true,449.093231,743.711792,124.917213,231.348541,41.741322,405.568481,116.737389,400.579163,402.870605,414.197357,593.222473,67.812241,774.547302,270.82489,151.641586,9.921497,79.526848,60.69376,323.64679,212.280869,0.738626,293.318054,112.637268,158.330185
2024-01-19 18:43:00,0.913732,300.276917,-59.512661,740.371399,1196.584717,409.585419,245.26799,46.510456,299.124207,229.778488,true,162.405075,209.928101,495.20816,210.010834,0.295982,199.41098,811.337769,true,54.190365,765.122742,162.915421,847.702759,196.19194,720.145935,225.592636,false,619.480774,271.960144,582.49231,577.097107,1104.501343,0.768411,590.453308,331.682678,81.939789,…,145.491608,254.698746,-24.335096,false,265.978333,223.539948,662.362915,97.795044,166.513611,266.232178,353.742035,181.154297,true,449.445831,742.737183,124.818642,231.301544,-84.950165,405.700806,116.770561,400.666962,402.791168,413.940277,592.911072,67.904289,774.92334,270.945374,151.443604,21.314808,79.659698,60.631454,323.524384,212.246246,0.827464,293.151001,112.717323,158.114243
2024-01-19 18:44:00,0.350472,299.8349,-8.34432,740.639771,1196.659058,409.46814,245.272186,46.432777,299.117706,229.788834,true,162.486069,210.025375,495.113831,210.251373,0.497201,199.566986,811.484741,true,53.849918,765.199341,162.554657,847.817627,196.498886,720.162292,225.764404,false,619.022949,271.745331,582.545044,577.133423,1104.705688,0.417721,590.250061,331.551178,81.976479,…,145.527008,254.502518,-19.054823,false,265.998444,223.450119,662.630981,97.693703,166.539291,266.702118,353.918121,180.697906,true,449.389465,743.598694,124.748535,232.460327,-12.434392,405

In [8]:
q.insert(df, "test")

In [9]:
q.fetch("select * from test")

time,ratio_5,process_17,deviation_2,process_61,process_48,process_64,process_83,process_41,process_50,process_63,binary_1,process_27,process_49,process_22,process_82,ratio_3,process_47,process_33,binary_2,process_29,process_67,process_15,process_76,process_11,process_18,process_37,binary_5,process_23,process_71,process_56,process_36,process_32,ratio_2,process_40,process_46,process_16,…,process_70,process_75,deviation_4,binary_4,process_9,process_21,process_4,process_34,process_62,process_42,process_68,process_14,binary_3,process_28,process_51,process_13,process_78,deviation_5,process_12,process_65,process_39,process_43,process_74,process_35,process_85,process_84,process_55,process_38,deviation_1,process_44,process_30,process_26,process_8,ratio_4,process_77,process_6,process_66
datetime[μs],f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,bool,f32,f32,f32,f32,f32,f32,f32,bool,f32,f32,f32,f32,f32,f32,f32,bool,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,bool,f32,f32,f32,f32,f32,f32,f32,f32,bool,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
2024-01-19 18:41:00,0.80593,300.455414,-7.731582,740.586548,1196.729736,409.302277,245.100708,46.619602,298.892365,230.019775,false,162.484207,210.038422,495.126068,210.16008,0.682768,199.832901,811.640747,true,54.126534,765.163574,162.938828,847.684875,196.484619,720.473694,225.642929,false,619.372437,271.541534,582.658813,576.870789,1104.865356,0.06206,590.350159,331.366028,82.230919,…,145.256012,254.71315,15.457435,false,265.88736,223.479507,662.40979,97.776443,166.551407,266.35614,353.419434,180.686722,true,449.556763,742.759705,124.45755,231.951065,-34.818947,405.773346,116.588463,401.022186,402.66922,414.300262,592.503967,67.806801,774.826721,270.66452,152.048203,18.754868,78.914421,60.698078,323.788177,212.264633,0.127276,293.387909,112.893295,158.082489
2024-01-19 18:42:00,0.452803,300.406464,-7.622905,740.469604,1196.512695,409.482239,245.651321,46.738731,299.173065,230.37178,false,162.648407,210.008469,495.072479,209.9673,0.218096,199.532593,811.500244,true,54.148956,765.016418,162.773697,847.629822,196.699417,720.023438,225.656479,false,620.314941,271.786346,582.508911,576.951965,1104.799438,0.237426,590.403198,331.131775,81.91906,…,145.129303,254.586044,11.870923,false,266.061432,223.640991,662.453491,97.772408,166.528839,265.902649,353.523163,180.776428,true,449.093231,743.711792,124.917213,231.348541,41.741322,405.568481,116.737389,400.579163,402.870605,414.197357,593.222473,67.812241,774.547302,270.82489,151.641586,9.921497,79.526848,60.69376,323.64679,212.280869,0.738626,293.318054,112.637268,158.330185
2024-01-19 18:43:00,0.913732,300.276917,-59.512661,740.371399,1196.584717,409.585419,245.26799,46.510456,299.124207,229.778488,true,162.405075,209.928101,495.20816,210.010834,0.295982,199.41098,811.337769,true,54.190365,765.122742,162.915421,847.702759,196.19194,720.145935,225.592636,false,619.480774,271.960144,582.49231,577.097107,1104.501343,0.768411,590.453308,331.682678,81.939789,…,145.491608,254.698746,-24.335096,false,265.978333,223.539948,662.362915,97.795044,166.513611,266.232178,353.742035,181.154297,true,449.445831,742.737183,124.818642,231.301544,-84.950165,405.700806,116.770561,400.666962,402.791168,413.940277,592.911072,67.904289,774.92334,270.945374,151.443604,21.314808,79.659698,60.631454,323.524384,212.246246,0.827464,293.151001,112.717323,158.114243
2024-01-19 18:44:00,0.350472,299.8349,-8.34432,740.639771,1196.659058,409.46814,245.272186,46.432777,299.117706,229.788834,true,162.486069,210.025375,495.113831,210.251373,0.497201,199.566986,811.484741,true,53.849918,765.199341,162.554657,847.817627,196.498886,720.162292,225.764404,false,619.022949,271.745331,582.545044,577.133423,1104.705688,0.417721,590.250061,331.551178,81.976479,…,145.527008,254.502518,-19.054823,false,265.998444,223.450119,662.630981,97.693703,166.539291,266.702118,353.918121,180.697906,true,449.389465,743.598694,124.748535,232.460327,-12.434392,405